tf.keras.layers.Conv2D 
- filters: 커널의 수
    - kernel_size : 커널의 크기
    - strides=(1, 1) : 몇칸을 이동할지
    - padding='valid','same'(0으로 모든부분 감싸기)

In [1]:
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D

from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

> Lenet-5 Model class

In [3]:
class Lenet(Sequential):
    def __init__(self, input_shape, np_classes):
        super().__init__()
        ## 데이터 전처리
        self.add(Conv2D(6, kernel_size=(5,5), strides=(1,1), padding='same', activation='tanh',input_shape=input_shape))
        self.add(AveragePooling2D(pool_size=(2, 2),strides=(2,2),padding='valid'))

        self.add(Conv2D(16, kernel_size=(5,5), strides=(1,1), padding='valid', activation='tanh'))
        self.add(AveragePooling2D(pool_size=(2, 2),strides=(2,2),padding='valid'))
        self.add(Flatten())
        self.add(Dense(120, activation='tanh'))
        ## 
        self.add(Dense(84, activation='tanh'))
        self.add(Dense(10, activation='softmax'))

        self.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        

In [4]:
from tensorflow.keras.datasets import mnist

(X_train,y_train),(X_test,y_test)= mnist.load_data()

In [5]:
# label one_hot encoding

y_train=to_categorical(y_train,10)
y_test=to_categorical(y_test,10)

In [6]:
X_train,X_test = X_train/255.0,X_test/255.0
X_train=X_train.reshape((60000,28,28,1))
X_test=X_test.reshape((10000,28,28,1))

In [7]:
model = Lenet(X_train[0].shape , 10)
model.summary()

Model: "lenet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 120)               48120 

In [8]:
log_dir= '/logs/fit/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
model.fit(X_train,y_train, epochs=20, validation_data=(X_test,y_test),verbose=1,callbacks=[tensorboard_callback])

Epoch 1/20
1875/1875 [==============================] - 18s 9ms/step - loss: 0.2273 - accuracy: 0.9317 - val_loss: 0.0991 - val_accuracy: 0.9669
Epoch 2/20
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0865 - accuracy: 0.9734 - val_loss: 0.0759 - val_accuracy: 0.9778
Epoch 3/20
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0584 - accuracy: 0.9824 - val_loss: 0.0613 - val_accuracy: 0.9804
Epoch 4/20
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0439 - accuracy: 0.9862 - val_loss: 0.0503 - val_accuracy: 0.9849
Epoch 5/20
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0371 - accuracy: 0.9881 - val_loss: 0.0506 - val_accuracy: 0.9829
Epoch 6/20
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0299 - accuracy: 0.9905 - val_loss: 0.0545 - val_accuracy: 0.9836
Epoch 7/20
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0259 - accuracy: 0.9916 - val_loss: 0.0514 - val_a

In [10]:
%load_ext tensorboard

In [12]:
%tensorboard --logdir logs/fit